# Create LLM Agent using OpenVINO and Qwen

LLM are limited to the knowledge on which they have been trained and the additional knowledge provided as context, as a result, if a useful piece of information is missing the provided knowledge, the model cannot “go around” and try to find it in other sources. This is the reason why we need to introduce the concept of Agents.

The core idea of agents is to use a language model to choose a sequence of actions to take. In agents, a language model is used as a reasoning engine to determine which actions to take and in which order. Agents can be seen as applications powered by LLMs and integrated with a set of tools like search engines, databases, websites, and so on. Within an agent, the LLM is the reasoning engine that, based on the user input, is able to plan and execute a set of actions that are needed to fulfill the request.

![agent](https://github.com/openvinotoolkit/openvino_notebooks/assets/91237924/22fa5396-8381-400f-a78f-97e25d57d807)

[Qwen-Agent](https://github.com/QwenLM/Qwen-Agent) is a framework for developing LLM applications based on the instruction following, tool usage, planning, and memory capabilities of Qwen. It also comes with example applications such as Browser Assistant, Code Interpreter, and Custom Assistant.

This notebook explores how to create a Function calling Agent step by step using OpenVINO and Qwen-Agent.

#### Table of contents:

- [Prerequisites](#Prerequisites)
- [Create a Function calling agent](#Create-a-Function-calling-agent)
  - [Create functions](#Create-functions)
  - [Download model](#Download-model)
  - [Select inference device for LLM](#Select-inference-device-for-LLM)
  - [Create LLM for Qwen-Agent](#Create-LLM-for-Qwen-Agent)
  - [Create Function-calling pipeline](#Create-Function-calling-pipeline)
- [Interactive Demo](#Interactive-Demo)
  - [Create tools](#Create-tools)
  - [Create AI agent demo with Qwen-Agent and Gradio UI](#Create-AI-agent-demo-with-Qwen-Agent-and-Gradio-UI)


## Prerequisites

[back to top ⬆️](#Table-of-contents:)


In [33]:
import os

os.environ["GIT_CLONE_PROTECTION_ACTIVE"] = "false"

%pip install -Uq pip
%pip uninstall -q -y optimum optimum-intel
%pip install --pre -Uq openvino openvino-tokenizers[transformers] --extra-index-url https://storage.openvinotoolkit.org/simple/wheels/nightly
%pip install -q --extra-index-url https://download.pytorch.org/whl/cpu\
"git+https://github.com/huggingface/optimum-intel.git"\
"git+https://github.com/openvinotoolkit/nncf.git"\
"git+https://github.com/QwenLM/Qwen-Agent.git"\
"torch>=2.1"\
"datasets"\
"accelerate"\
"transformers>=4.38.1" "modelscope_studio"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


## Create a Function calling agent

[back to top ⬆️](#Table-of-contents:)

Function calling allows a model to detect when one or more tools should be called and respond with the inputs that should be passed to those tools. In an API call, you can describe tools and have the model intelligently choose to output a structured object like JSON containing arguments to call these tools. The goal of tools APIs is to more reliably return valid and useful tool calls than what can be done using a generic text completion or chat API.

We can take advantage of this structured output, combined with the fact that you can bind multiple tools to a tool calling chat model and allow the model to choose which one to call, to create an agent that repeatedly calls tools and receives results until a query is resolved.

### Create a function

[back to top ⬆️](#Table-of-contents:)

First, we need to create a example function/tool for getting the informantion of current weahter.

In [1]:
import requests

def get_current_weather(city_name):
    """Get the current weather in a given city name"""
    if not isinstance(city_name, str):
        raise TypeError("City name must be a string")
    key_selection = {
        "current_condition": [
            "temp_C",
            "FeelsLikeC",
            "humidity",
            "weatherDesc",
            "observation_time",
        ],
    }
    resp = requests.get(f"https://wttr.in/{city_name}?format=j1")
    resp.raise_for_status()
    resp = resp.json()
    ret = {k: {_v: resp[k][0][_v] for _v in v} for k, v in key_selection.items()}

    return str(ret)

Wrap the function's name and decription into a json list, and it will help LLM to find out which function should be called for current task.

In [2]:
functions = [{
    'name': 'get_current_weather',
    'description': 'Get the current weather in a given city name',
    'parameters': {
        'type': 'object',
        'properties': {
            'city_name': {
                'type': 'string',
                'description': 'The city and state, e.g. San Francisco, CA',
            },
        },
        'required': ['city_name'],
    },
}]

### Download model

[back to top ⬆️](#Table-of-contents:)

Large Language Models (LLMs) are a core component of Agent. In this example, we will demostrate how to create a OpenVINO LLM model in Qwen-Agent framework. Since Qwen2 can support function calling during text generation, we select `Qwen/Qwen2-7B-Instruct` as LLM in agent pipeline.

* **Qwen/Qwen2-7B-Instruct** - Qwen2 is the new series of Qwen large language models. Compared with the state-of-the-art opensource language models, including the previous released Qwen1.5, Qwen2 has generally surpassed most opensource models and demonstrated competitiveness against proprietary models across a series of benchmarks targeting for language understanding, language generation, multilingual capability, coding, mathematics, reasoning, etc. [Model Card](https://huggingface.co/Qwen/Qwen2-7B-Instruct)

To run LLM locally, we have to download the model in the first step. It is possible to [export your model](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#export) to the OpenVINO IR format with the CLI, and load the model from local folder.


In [3]:
from pathlib import Path

model_id = "Qwen/Qwen2-7B-Instruct"
model_path = "Qwen2-7B-Instruct-ov"

if not Path(model_path).exists():
    !optimum-cli export openvino --model {model_id} --task text-generation-with-past --trust-remote-code --weight-format int4 {model_path}

### Select inference device for LLM

[back to top ⬆️](#Table-of-contents:)


In [4]:
import openvino as ov
import ipywidgets as widgets

core = ov.Core()

support_devices = core.available_devices
if "NPU" in support_devices:
    support_devices.remove("NPU")

device = widgets.Dropdown(
    options=support_devices + ["AUTO"],
    value="CPU",
    description="Device:",
    disabled=False,
)

device

Dropdown(description='Device:', options=('CPU', 'GPU', 'AUTO'), value='CPU')

### Create LLM for Qwen-Agent

[back to top ⬆️](#Table-of-contents:)

OpenVINO has been integrated into the `Qwen-Agent` framework. You can use following method to create a OpenVINO based LLM for a `Qwen-Agent` pipeline.

In [5]:
from qwen_agent.llm import get_chat_model

ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}
llm_cfg = {
    'ov_model_dir': model_path,
    'model_type': 'openvino',
    'device': device.value,
    'ov_config': ov_config
}
llm = get_chat_model(llm_cfg)

2024-06-26 20:54:08.019761: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-26 20:54:08.022283: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-26 20:54:08.053977: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-26 20:54:08.054002: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-26 20:54:08.054028: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

You can get additional inference speed improvement with [Dynamic Quantization of activations and KV-cache quantization] on CPU(https://docs.openvino.ai/2024/learn-openvino/llm_inference_guide/llm-inference-hf.html#enabling-openvino-runtime-optimizations). These options can be enabled with `ov_config` as follows:

## Create Function-calling pipeline

[back to top ⬆️](#Table-of-contents:)

After defining the functions and LLM, we can build the agent pipeline with capability of function calling.

The workflow of Qwen2 function calling consists of several steps:

1. Role `user` sending the request.
2. Check if the model wanted to call a function, and call the function if needed
3. Get the observation from `function`'s results.
4. Consolidate the observation into final respone of `assistant`.

A typical multi-turn dialogue structure is as follows:

```
{'role': 'user', 'content': 'create a picture of cute cat'},
{'role': 'assistant', 'content': '', 'function_call': {'name': 'my_image_gen', 'arguments': '{"prompt": "a cute cat"}'}},
{'role': 'function', 'content': '{"image_url": "https://image.pollinations.ai/prompt/a%20cute%20cat"}', 'name': 'my_image_gen'},
{'role': 'assistant', 'content': "Here is the image of a cute cat based on your description:\n\n![](https://image.pollinations.ai/prompt/a%20cute%20cat)."},
    
```

In [6]:
import json

print('# User question:')
messages = [{'role': 'user', 'content': "What's the weather like in San Francisco?"}]
print(messages)

print('# Assistant Response 1:')
responses = []

# Step 1: Role `user` sending the request
for responses in llm.chat(
        messages=messages,
        functions=functions,
        stream=False,
):
    print(responses)

messages.append(responses)

# Step 2: check if the model wanted to call a function, and call the function if needed
last_response = messages[-1]
if last_response.get('function_call', None):
    available_functions = {
        'get_current_weather': get_current_weather,
    }  # only one function in this example, but you can have multiple
    function_name = last_response['function_call']['name']
    function_to_call = available_functions[function_name]
    function_args = json.loads(last_response['function_call']['arguments'])
    function_response = function_to_call(
        city_name=function_args.get('city_name'),
    )
    print('# Function Response:')
    print(function_response)

    # Step 3: Get the observation from `function`'s results
    messages.append({
        'role': 'function',
        'name': function_name,
        'content': function_response,
    })

    print('# Assistant Response 2:')
    # Step 4: Consolidate the observation from function into final respone
    for responses in llm.chat(
            messages=messages,
            functions=functions,
            stream=False,
    ):  # get a new response from the model where it can see the function response
        print(responses)

# User question:
[{'role': 'user', 'content': "What's the weather like in San Francisco?"}]
# Assistant Response 1:
{'role': 'assistant', 'content': '', 'function_call': {'name': 'get_current_weather', 'arguments': '{"city_name": "San Francisco, CA"}'}}
# Function Response:
{'current_condition': {'temp_C': '21', 'FeelsLikeC': '21', 'humidity': '69', 'weatherDesc': [{'value': 'Clear'}], 'observation_time': '02:22 AM'}}
# Assistant Response 2:
{'role': 'assistant', 'content': 'The current weather in San Francisco is Clear. The temperature is 21°C and it feels like the same temperature due to low humidity effects. The humidity level is at 69%.'}


## Interactive Demo

[back to top ⬆️](#Table-of-contents:)

Let's create a interactive agent using [Gradio](https://www.gradio.app/).


### Create tools

[back to top ⬆️](#Table-of-contents:)

Qwen-Agent provides a mechanism for [registering tools](https://github.com/QwenLM/Qwen-Agent/blob/main/docs/tool.md). For example, to register your own image generation tool:

- Specify the tool’s name, description, and parameters. Note that the string passed to `@register_tool('my_image_gen')` is automatically added as the `.name` attribute of the class and will serve as the unique identifier for the tool.
- Implement the `call(...)` function.

In [7]:
import urllib.parse
import json5
from qwen_agent.tools.base import BaseTool, register_tool

@register_tool('image_generation')
class ImageGeneration(BaseTool):
    description = 'AI painting (image generation) service, input text description, and return the image URL drawn based on text information.'
    parameters = [{
        'name': 'prompt',
        'type': 'string',
        'description': 'Detailed description of the desired image content, in English',
        'required': True
    }]

    def call(self, params: str, **kwargs) -> str:
        prompt = json5.loads(params)['prompt']
        prompt = urllib.parse.quote(prompt)
        return json5.dumps(
            {'image_url': f'https://image.pollinations.ai/prompt/{prompt}'},
            ensure_ascii=False)


@register_tool('get_current_weather')
class GetCurrentWeather(BaseTool):
    description = 'Get the current weather in a given city name.'
    parameters = [{
        'name': 'city_name',
        'type': 'string',
        'description': 'The city and state, e.g. San Francisco, CA',
        'required': True
    }]

    def call(self, params: str, **kwargs) -> str:
        # `params` are the arguments generated by the LLM agent.
        city_name = json5.loads(params)['city_name']
        key_selection = {
            "current_condition": [
                "temp_C",
                "FeelsLikeC",
                "humidity",
                "weatherDesc",
                "observation_time",
            ],
        }
        resp = requests.get(f"https://wttr.in/{city_name}?format=j1")
        resp.raise_for_status()
        resp = resp.json()
        ret = {k: {_v: resp[k][0][_v] for _v in v} for k, v in key_selection.items()}
        return str(ret)


In [8]:
tools = ['image_generation', 'get_current_weather']

### Create AI agent demo with Qwen-Agent and Gradio UI

[back to top ⬆️](#Table-of-contents:)

The Agent class serves as a higher-level interface for Qwen-Agent, where an Agent object integrates the interfaces for tool calls and LLM (Large Language Model). The Agent receives a list of messages as input and produces a generator that yields a list of messages, effectively providing a stream of output messages.

Qwen-Agent offer a generic Agent class: the `Assistant` class, which, when directly instantiated, can handle the majority of Single-Agent tasks. Features:

- It supports role-playing;
- It provides automatic planning and tool calls abilities;
- RAG (Retrieval-Augmented Generation): It accepts documents input, and can use an integrated RAG strategy to parse the documents.

In [9]:
from qwen_agent.agents import Assistant
from qwen_agent.gui import WebUI

bot = Assistant(llm=llm_cfg,
                function_list=tools,)

# chatbot_config = {
#     'user.name': 'OpenVINO Agent',
#     'prompt.suggestions': ["hello"]
# }
# demo = WebUI(
#     bot,
#     chatbot_config=chatbot_config,
# )


# # if you are launching remotely, specify server_name and server_port
# #  demo.run(server_name='your server name', server_port='server port in int')
# demo.run(server_name = '10.3.233.99', server_port = 4569)

Compiling the model to CPU ...
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [33]:
from typing import List, Optional, Union
from qwen_agent.llm.schema import CONTENT, FILE, IMAGE, NAME, ROLE, USER, Message
from qwen_agent.gui.utils import convert_fncall_to_text, convert_history_to_chatbot, get_avatar_image

class OpenVINOUI(WebUI):
    
    def request_cancel(self):
        self.agent_list[0].llm.ov_model.request.cancel()

    def run(self,
        messages: List[Message] = None,
        share: bool = False,
        server_name: str = None,
        server_port: int = None,
        concurrency_limit: int = 10,
        enable_mention: bool = False,
        **kwargs):
        self.run_kwargs = kwargs
    
        from qwen_agent.gui.gradio import gr, mgr
    
        customTheme = gr.themes.Default(
            primary_hue=gr.themes.utils.colors.blue,
            radius_size=gr.themes.utils.sizes.radius_none,
        )
    
        with gr.Blocks(
            theme=gr.themes.Soft(),
            css=".disclaimer {font-variant-caps: all-small-caps;}",
        ) as self.demo:
            history = gr.State([])
    
            with gr.Row():
                chatbot = mgr.Chatbot(
                    value=convert_history_to_chatbot(messages=messages),
                    avatar_images=[
                        self.user_config,
                        self.agent_config_list,
                    ],
                    height=900,
                    avatar_image_width=80,
                    flushing=False,
                    show_copy_button=True,
                )

            with gr.Row():
                with gr.Column(scale=8):
                    input = mgr.MultimodalInput(placeholder=self.input_placeholder, submit_button_props={"min_width": 320})
                with gr.Column(scale=1, min_width=160):
                    stop = gr.Button("Stop")
                    clear = gr.Button("Clear")
                # input = gr.MultimodalTextbox(interactive=True, placeholder="Enter message or upload file...", show_label=False)

                # input = gr.Textbox(
                #     label="Chat Message Box",
                #     placeholder="Chat Message Box",
                #     show_label=False,
                #     container=False,
                # )
            with gr.Row():
                gr.Examples(["draw a dog"] , inputs=[input], label="Click on any example and press the 'Submit' button")
            
            input_promise = input.submit(
                fn=self.add_text,
                inputs=[input, chatbot, history],
                outputs=[input, chatbot, history],
                queue=False,
            )
            input_promise.then(self.flushed, None, [input])
    
            self.demo.load(None)
            
            input_promise = input_promise.then(
                self.agent_run,
                [chatbot, history],
                [chatbot, history],
            )
            stop.click(
                fn=self.request_cancel,
                inputs=None,
                outputs=None,
                cancels=[input_promise],
                queue=False,
            )
            clear.click(lambda: None, None, chatbot, queue=False)
    
        self.demo.launch(share=share,server_name=server_name,server_port=server_port)
    def close():
        self.demo.close
    
    
chatbot_config = {
    'user.name': 'OpenVINO Agent',
    'prompt.suggestions': ["hello"]
}
demo = OpenVINOUI(
    bot,
    chatbot_config=chatbot_config,
)
demo.run(server_name = '10.3.233.99', server_port = 4544)

Running on local URL:  http://10.3.233.99:4544
IMPORTANT: You are using gradio version 4.21.0, however version 4.29.0 is available, please upgrade.
--------


ValueError: When localhost is not accessible, a shareable link must be created. Please set share=True or check your proxy settings to allow access to localhost.